In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from geopy import distance
import matplotlib.pyplot as plt
import re
pd.set_option('max.columns', 2000)

In [2]:
# set path to github directory
os.chdir('..')

# set path to raw data folder
github_path = Path(os.getcwd())
data_path = github_path / 'data' / 'raw'

In [12]:
# load data frames
airbnb = pd.read_csv(data_path / 'train.csv')

In [13]:
def handle_entry(s):
    # remove curly brace
    s = s.strip('{').strip('}')
    s = s.lower()
    # split
    s_list = s.split(",")
    # remove extra quote
    for i, curr_s in enumerate(s_list):
        s_list[i] = s_list[i].strip('"')
    return s_list

In [14]:
airbnb['amenities'] = airbnb['amenities'].apply(lambda x: handle_entry(x))

In [15]:
amenity_dict = {'amenity_tv': ['cable tv','tv'], 
 'amenity_baby': ['baby bath','baby monitor', 'babysitter recommendations', 'changing table', 
                  "children’s books and toys", "children’s dinnerware", 'family/kid friendly',
                  'fireplace guards', 'crib', 'high chair', "pack ’n play/travel crib", 'table corner guards',
                  'outlet covers', 'stair gates'],
'amenity_bed': ['bed linens', "extra pillows and blankets",'firm mattress','firm matress','firm mattress'],
'amenity_accessibility': [' smooth pathway to front door','accessible-height bed','accessible-height toilet',
                          'disabled parking spot','fixed grab bars for shower & toilet','flat smooth pathway to front door',
                          'grab-rails for shower and toilet','ground floor access','handheld shower head',
                          'path to entrance lit at night','roll-in shower with chair','step-free access',
                          'well-lit path to entrance','wheelchair accessible','wide clearance to bed',
                          'wide clearance to shower & toilet','wide clearance to shower and toilet','wide doorway',
                          'wide entryway','wide hallway clearance'],
'amenity_aircondition': ['air conditioning'],
'amenity_bbqgrill': ['bbq grill'], 
'amenity_breakfast': ['breakfast'],
'amenity_buzzer_wireless_intercom':['buzzer/wireless intercom'],
'amenity_cleaning_bf_checkout': ['cleaning before checkout'],
'amenity_pets': ['cat(s)', 'dog(s)', 'other pet(s)', 'pets allowed', 'pets live on this property'],
'amenity_washer_dryer':['dryer','washer','washer / dryer'],
'amenity_ev_charger':['ev charger'],
'amenity_elevator':['elevator','elevator in building'],
'amenity_free_park':['free parking on premises','free parking on street'],
'amenity_game':['game console'],
'amenity_indoor_fireplace': ['indoor fireplace'],
'amenity_gym':['gym'],
'amenity_clothes':['iron', 'hangers'],
'amenity_essential':['essentials'],
'amenity_hot_water': ['hot water'],
'amenity_garden_backyard':['garden or backyard'],
'amenity_bathroom':['bath towel', 'hand or paper towel', 'toilet paper', 'hand soap', 'body soap', 
                    'bathtub', 'shampoo', 'bathtub with shower chair', 'hot tub', 'shampoo', 'hair dryer'],
'amenity_host_greet': ['host greets you'], 
'amenity_heating': ['heating'],
'amenity_kitchen': ['refrigerator', 'dishes and silverware', 'microwave', 'hot water kettle', 
                    'dishwasher', 'coffee maker', 'stove', 'oven', 'cooking basics'],
'amenity_lake': ['lake access'],
'amenity_work': ['laptop friendly workspace'],
'amenity_bedroom_lock': ['lock on bedroom door'],
'amenity_lockbox': ['lockbox', 'keypad'],
'amenity_longterm_stays': ['long term stays allowed'],
'amenity_luggage_dropoff': ['luggage dropoff allowed'],
'amenity_other': ['other'],
'amenity_paid_parking': ['paid parking off premises'],
'amenity_patio_balcony': ['patio or balcony'],
'amenity_wifi': ['pocket wifi', 'wireless internet', 'internet', 'ethernet connection', 'internet'],
'amenity_pool': ['pool'],
'amenity_privacy': ['private bathroom', 'private entrance', 'private living room', 'room-darkening shades'],
'amenity_safety': ['safety card', 'smart lock', 'smartlock', 'smoke detector', 
                   'fire extinguisher', 'first aid kit', 'window guards'],
'amenity_self_checkin': ['self check-in'],
'amenity_single_level_home': ['single level home'],
'amenity_smoking_allowed': ['smoking allowed'],
'amenity_suitable_for_events': ['suitable for events'],
'amenity_waterfront': ['waterfront'],
'amenity_window_guards': ['window guards'],
'amenity_self_checkin':['self check-in'],
'amenity_beach_essentials':['beach essentials'],
'amenity_beach_front':['beachfront'],
'amenity_doorman':['doorman','doorman entry'],
'amenity_24hour_checkin':['24-hour check-in']}

In [16]:
def ohe_amenities(amenity, x):
    if amenity in x:
        return 1
    else:
        return 0

In [17]:
for agroup in amenity_dict.keys():
    amenity_cols = amenity_dict[agroup]
    for amenity in amenity_cols:
        airbnb[amenity] = airbnb['amenities'].apply(lambda x: ohe_amenities(amenity, x))
    airbnb[agroup] = airbnb[amenity_cols].sum(axis=1)
    airbnb = airbnb.drop(columns=amenity_cols)
    airbnb[agroup] = np.where(airbnb[agroup] >= 1, 1, 0)

In [18]:
airbnb.describe()

,id,log_price,accommodates,bathrooms,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds,amenity_tv,amenity_baby,amenity_bed,amenity_accessibility,amenity_aircondition,amenity_bbqgrill,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cleaning_bf_checkout,amenity_pets,amenity_washer_dryer,amenity_ev_charger,amenity_elevator,amenity_free_park,amenity_game,amenity_indoor_fireplace,amenity_gym,amenity_clothes,amenity_essential,amenity_hot_water,amenity_garden_backyard,amenity_bathroom,amenity_host_greet,amenity_heating,amenity_kitchen,amenity_lake,amenity_work,amenity_bedroom_lock,amenity_lockbox,amenity_longterm_stays,amenity_luggage_dropoff,amenity_other,amenity_paid_parking,amenity_patio_balcony,amenity_wifi,amenity_pool,amenity_privacy,amenity_safety,amenity_self_checkin,amenity_single_level_home,amenity_smoking_allowed,amenity_suitable_for_events,amenity_waterfront,amenity_window_guards,amenity_beach_essentials,amenity_beach_front,amenity_doorman,amenity_24hour_checkin
count,7.411100e+04,74111.000000,74111.000000,73911.000000,74111.000000,74111.000000,74111.000000,57389.000000,74020.000000,73980.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000,74111.000000
mean,1.126662e+07,4.782069,3.155146,1.235263,38.445958,-92.397525,20.900568,94.067365,1.265793,1.710868,0.723995,0.503758,0.059303,0.081216,0.744964,0.008541,0.112075,0.229831,0.003657,0.239101,0.590331,0.000661,0.232584,0.319777,0.006463,0.125487,0.101078,0.725169,0.863637,0.057576,0.015234,0.793216,0.016138,0.905034,0.063918,0.000243,0.589697,0.242650,0.115462,0.022547,0.022129,0.002942,0.000094,0.016462,0.966483,0.084778,0.136970,0.850063,0.148979,0.007003,0.049561,0.057589,0.001552,0.008298,0.001376,0.000445,0.063014,0.256575
std,6.081735e+06,0.717394,2.153589,0.582044,3.080167,21.705322,37.828641,7.836556,0.852143,1.254142,0.447022,0.499989,0.236192,0.273168,0.435885,0.092024,0.315461,0.420727,0.060360,0.426537,0.491776,0.025705,0.422482,0.466393,0.080135,0.331273,0.301434,0.446432,0.343176,0.232941,0.122483,0.405002,0.126007,0.293169,0.244608,0.015583,0.491892,0.428688,0.319581,0.148456,0.147104,0.054156,0.009718,0.127244,0.179984,0.278553,0.343818,0.357012,0.356070,0.083391,0.217037,0.232967,0.039362,0.090717,0.037073,0.021097,0.242989,0.436746
min,3.440000e+02,0.000000,1.000000,0.000000,33.338905,-122.511500,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.261964e+06,4.317488,2.000000,1.000000,34.127908,-118.342374,1.000000,92.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.225415e+07,4.709530,2.000000,1.000000,40.662138,-76.996965,6.000000,96.000000,1.0

In [19]:
airbnb.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,amenity_tv,amenity_baby,amenity_bed,amenity_accessibility,amenity_aircondition,amenity_bbqgrill,amenity_breakfast,amenity_buzzer_wireless_intercom,amenity_cleaning_bf_checkout,amenity_pets,amenity_washer_dryer,amenity_ev_charger,amenity_elevator,amenity_free_park,amenity_game,amenity_indoor_fireplace,amenity_gym,amenity_clothes,amenity_essential,amenity_hot_water,amenity_garden_backyard,amenity_bathroom,amenity_host_greet,amenity_heating,amenity_kitchen,amenity_lake,amenity_work,amenity_bedroom_lock,amenity_lockbox,amenity_longterm_stays,amenity_luggage_dropoff,amenity_other,amenity_paid_parking,amenity_patio_balcony,amenity_wifi,amenity_pool,amenity_privacy,amenity_safety,amenity_self_checkin,amenity_single_level_home,amenity_smoking_allowed,amenity_suitable_for_events,amenity_waterfront,amenity_window_guards,amenity_beach_essentials,amenity_beach_front,amenity_doorman,amenity_24hour_checkin
0,6901257,5.010635,Apartment,Entire home/apt,"[wireless internet, air conditioning, kitchen,...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,6304928,5.129899,Apartment,Entire home/apt,"[wireless internet, air conditioning, kitchen,...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,7919400,4.976734,Apartment,Entire home/apt,"[tv, cable tv, wireless internet, air conditio...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,13418779,6.620073,House,Entire home/apt,"[tv, cable tv, internet, wireless internet, ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,3808709,4.744932,Apartment,Entire home/apt,"[tv, internet, wireless internet, air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,14549287,4.605170,Apartment,Private room,[],1,1.0,Real Bed,flexible,False,NYC,one room in bushwick apt,NaN,t,t,NaN,2013-03-24,f,NaN,40.709025,-73.939405,one room bushwick,Williamsburg,0,NaN,https://a0